In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly as py
import plotly.graph_objs as go
import requests
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.utils import *
from datetime import datetime
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
import random
import re

Using TensorFlow backend.


In [2]:
file = 'last_of_us.txt'

text = ''
array = []
with open(file, 'r', encoding='UTF-8')  as f:
    lines = f.readlines()
    
    text = text.join([i for i in lines if re.match(r'^[A-Z].*:', i)])

for t in text.split('\n')[:1000]:
    array.append(t)

# 전체에 배열을 뒤져서  겹치지 않는 단어별로 차례대로 숫자를 맥이고 각가의 단어를 숫자로 치환한다..
token = Tokenizer(lower=False, filters='.,?;\'\"-')
token.fit_on_texts(array)
token.word_index  

{'Joel:': 1,
 'Tess:': 2,
 'I': 3,
 's': 4,
 'the': 5,
 'you': 6,
 'Ellie:': 7,
 'to': 8,
 'it': 9,
 't': 10,
 'we': 11,
 're': 12,
 'a': 13,
 'You': 14,
 'this': 15,
 'that': 16,
 'Sarah:': 17,
 'We': 18,
 'of': 19,
 'here': 20,
 'up': 21,
 'out': 22,
 'on': 23,
 'What': 24,
 'get': 25,
 'is': 26,
 'are': 27,
 'me': 28,
 'Tommy:': 29,
 'Let': 30,
 'your': 31,
 'there': 32,
 'm': 33,
 'do': 34,
 'in': 35,
 'go': 36,
 'Oh': 37,
 'they': 38,
 'for': 39,
 'just': 40,
 'Yeah': 41,
 'us': 42,
 'Marlene:': 43,
 'can': 44,
 'It': 45,
 'They': 46,
 'know': 47,
 'with': 48,
 'what': 49,
 'll': 50,
 'Guard:': 51,
 '2:': 52,
 'No': 53,
 'and': 54,
 'Tess': 55,
 'was': 56,
 'gonna': 57,
 'Hey': 58,
 'not': 59,
 'be': 60,
 'How': 61,
 'Joel': 62,
 'way': 63,
 'back': 64,
 'Alright': 65,
 'There': 66,
 'got': 67,
 'see': 68,
 'That': 69,
 'mon': 70,
 'around': 71,
 'don': 72,
 'C': 73,
 'shit': 74,
 'through': 75,
 'This': 76,
 'all': 77,
 'Okay': 78,
 'Look': 79,
 'have': 80,
 'Get': 81,
 'hell': 8

In [3]:
#단어의 갯수
len(token.word_index)

1384

In [4]:
#array있는 String을 seq로 변경한다. (위에 단어장에서)
seq = token.texts_to_sequences(array)

# RNN 입력은 3차원으로 한다.
#(Sample, timestep, feature)
# i am student

In [5]:
# 길이가 부족한것들은 0으로 채워준다.
# 길이가 넘치는것은 뒤에서부터 채우고 버린다.
seq = pad_sequences(seq, maxlen=10)
seq

array([[ 130,  626,    8, ...,  627,  104,    3],
       [   0,    0,    0, ...,    0,   17,   58],
       [   0,    0,    0, ...,    0,    1,  628],
       ...,
       [  57, 1382,   22, ...,   21,   55,  418],
       [   0,    0,    0, ...,    0,    7,  328],
       [   0,    0,    0, ...,    1, 1384,    9]])

In [6]:
X = seq

In [7]:
#X의 맨위에서 한칸을 제거하고 맨밑으로 넣는다. (바로 다음것을 가져오기위해 루프)
Y = np.vstack((X[1:], X[0]))
Y

array([[   0,    0,    0, ...,    0,   17,   58],
       [   0,    0,    0, ...,    0,    1,  628],
       [   0,    0,    0, ...,  111,  266,  331],
       ...,
       [   0,    0,    0, ...,    0,    7,  328],
       [   0,    0,    0, ...,    1, 1384,    9],
       [ 130,  626,    8, ...,  627,  104,    3]])

In [8]:
Y.shape

(1000, 10)

In [9]:
X = X.reshape(-1, 10, 1)
Y = Y.reshape(-1, 10 ,1)

X

array([[[ 130],
        [ 626],
        [   8],
        ...,
        [ 627],
        [ 104],
        [   3]],

       [[   0],
        [   0],
        [   0],
        ...,
        [   0],
        [  17],
        [  58]],

       [[   0],
        [   0],
        [   0],
        ...,
        [   0],
        [   1],
        [ 628]],

       ...,

       [[  57],
        [1382],
        [  22],
        ...,
        [  21],
        [  55],
        [ 418]],

       [[   0],
        [   0],
        [   0],
        ...,
        [   0],
        [   7],
        [ 328]],

       [[   0],
        [   0],
        [   0],
        ...,
        [   1],
        [1384],
        [   9]]])

In [10]:
X.shape

(1000, 10, 1)

In [11]:
Y = to_categorical(Y)
Y.shape

(1000, 10, 1385)

In [12]:
'''
LSTM에서는
-> -> -> timestep은 기본적으로 오른쪽으로 전달한다.

Bidirectional
<- <- <- <- 양방향으로 전달한다.
'''

model = Sequential()
#input_shape (timestep, feature))
model.add(Bidirectional(SimpleRNN(128, return_sequences=True), input_shape=(10,1)))
model.add(Bidirectional(SimpleRNN(128, return_sequences=True)))
model.add(Dense(1385, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 10, 256)           33280     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 10, 256)           98560     
_________________________________________________________________
dense_1 (Dense)              (None, 10, 1385)          355945    
Total params: 487,785
Trainable params: 487,785
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X,Y,epochs=100, batch_size=1)


Epoch 1/100
1000/1000 [==============================] - 12s 12ms/step - loss: 5.0091 - accuracy: 0.3235
Epoch 2/100
1000/1000 [==============================] - 10s 10ms/step - loss: 4.6523 - accuracy: 0.3265
Epoch 3/100
1000/1000 [==============================] - 10s 10ms/step - loss: 4.4889 - accuracy: 0.3272
Epoch 4/100
1000/1000 [==============================] - 10s 10ms/step - loss: 4.3414 - accuracy: 0.3257
Epoch 5/100
1000/1000 [==============================] - 10s 10ms/step - loss: 4.1927 - accuracy: 0.3260
Epoch 6/100
1000/1000 [==============================] - 10s 10ms/step - loss: 4.0351 - accuracy: 0.3264
Epoch 7/100
1000/1000 [==============================] - 10s 10ms/step - loss: 3.8884 - accuracy: 0.3255
Epoch 8/100
1000/1000 [==============================] - 11s 11ms/step - loss: 3.7439 - accuracy: 0.3264
Epoch 9/100
1000/1000 [==============================] - 11s 11ms/step - loss: 3.5957 - accuracy: 0.3285
Epoch 10/100
1000/1000 [==============================]

In [ ]:
model.save('lasOfUs.model')